In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization, Dropout, Bidirectional
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [ ]:
file_path = 'output_predictions.csv'
data = pd.read_csv(file_path)

#Check unique values in each column
for column in data.columns:
    unique_values = data[column].unique()
    print(f"Unique values in {column}: {unique_values}")


Unique values in left_elbow_angle: [ 38.78  20.98  38.61 ...  92.48  88.52 102.17]
Unique values in left_knee_angle: [  4.33   6.31   6.46 ... 144.14 141.82  46.85]
Unique values in left_shoulder_angle: [166.62 112.02 164.55 ...  73.01   5.69  81.76]
Unique values in left_hip_angle: [  6.1    5.24   6.86 ... 118.52  51.17  37.2 ]
Unique values in right_elbow_angle: [16.91 11.36 15.34 ... 95.08 86.8  80.85]
Unique values in right_knee_angle: [ 6.22  8.41  5.32 ... 63.25 53.42 58.69]
Unique values in right_shoulder_angle: [160.43  81.24 155.89 ...  76.98  71.78  62.59]
Unique values in right_hip_angle: [12.64 16.02 11.93 ... 50.69 37.2  39.08]
Unique values in predictions: ['[0]' '[1]' '[10]' '[2]']


In [ ]:
moves = data.iloc[:, -1].apply(lambda x: int(x.strip('[]'))).values

# Separate moves of F1 and F2
fighter1_moves = moves[1::2]
fighter2_moves = moves[2::2]

# Ensure arrays have the same length!
min_length = min(len(fighter1_moves), len(fighter2_moves))
fighter1_moves = fighter1_moves[:min_length]
fighter2_moves = fighter2_moves[:min_length]
# Sequences n labels...
sequences = []
labels = []

for i in range(len(fighter1_moves) - 3):
    sequences.append(fighter1_moves[i:i+3])  # Append a 3-move sequence from F1...
    labels.append(fighter2_moves[i+3])       # Append the next move from F2...

# Convert to np arrays
sequences = np.array(sequences)
labels = np.array(labels)

In [ ]:
# Encode moves into integers
label_encoder = LabelEncoder()
sequences = label_encoder.fit_transform(sequences.ravel()).reshape(sequences.shape)
labels = label_encoder.fit_transform(labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2, random_state=69)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train.reshape(X_train.shape[0], -1), y_train)

# KNN Predictions
knn_train_predictions = knn.predict(X_train.reshape(X_train.shape[0], -1)).reshape(-1, 1)
knn_test_predictions = knn.predict(X_test.reshape(X_test.shape[0], -1)).reshape(-1, 1)


In [ ]:
# Combine KNN predictions with LSTM input
X_train_combined = np.concatenate([X_train, knn_train_predictions], axis=1)
X_test_combined = np.concatenate([X_test, knn_test_predictions], axis=1)

# Reshape for LSTM input
X_train_combined = X_train_combined.reshape((X_train_combined.shape[0], X_train_combined.shape[1], 1))
X_test_combined = X_test_combined.reshape((X_test_combined.shape[0], X_test_combined.shape[1], 1))

In [ ]:
# Create tf dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_combined, y_train))

# Calc class dis
class_counts = np.bincount(y_train)
class_weights = np.array([max(class_counts) / count for count in class_counts])

# Assign sample weights based on the class distribution!
def assign_sample_weights(X, y):
    weight = tf.gather(class_weights, y)
    return X, y, weight

# Apply
weighted_dataset = train_dataset.map(assign_sample_weights)

# Separate into three components (features, labels, weights) VITAL
def unpack(X, y, weight):
    return (X, y), weight

weighted_dataset = weighted_dataset.map(unpack)

# Resample dataset, balance classes
resampled_dataset = weighted_dataset.flat_map(
    lambda data, weight: tf.data.Dataset.from_tensors(data).repeat(tf.cast(weight, tf.int64))
)

# Shuffle, batch prefetch
balanced_dataset = resampled_dataset.shuffle(buffer_size=len(y_train)).batch(1024).prefetch(tf.data.AUTOTUNE)


In [ ]:
model = Sequential([
    tf.keras.layers.Input(shape=(X_train_combined.shape[1], 1)),  # Shape (timesteps, features)!!!
    
    # LSTM
    Bidirectional(LSTM(64, return_sequences=True)),
    BatchNormalization(),
    Dropout(0.3),

    LSTM(128, return_sequences=True),
    BatchNormalization(),
    Dropout(0.3),

    LSTM(64, return_sequences=False),
    BatchNormalization(),
    Dropout(0.3),

    # Fully connected
    Dense(128, activation='relu', kernel_regularizer=l2(0.002)),
    BatchNormalization(),
    Dropout(0.5),

    Dense(64, activation='relu', kernel_regularizer=l2(0.002)),
    BatchNormalization(),
    Dropout(0.5),

    Dense(32, activation='relu', kernel_regularizer=l2(0.002)),
    BatchNormalization(),

    # Output
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.0003)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with balanced batches!!!
model.fit(balanced_dataset, epochs=100, validation_data=(X_test_combined, y_test), callbacks=[reduce_lr, early_stopping])

# Eval
test_loss, test_acc = model.evaluate(X_test_combined, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

model.save('kickboxing_predictor_improved_with_knn.keras')

Epoch 1/100
    226/Unknown 22s 47ms/step - accuracy: 0.5171 - loss: 1.5670

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


228/228 ━━━━━━━━━━━━━━━━━━━━ 26s 65ms/step - accuracy: 0.5172 - loss: 1.5656 - val_accuracy: 0.6513 - val_loss: 1.2427 - learning_rate: 0.0010
Epoch 2/100
228/228 ━━━━━━━━━━━━━━━━━━━━ 16s 60ms/step - accuracy: 0.5839 - loss: 1.2024 - val_accuracy: 0.6479 - val_loss: 1.1526 - learning_rate: 0.0010
Epoch 3/100
228/228 ━━━━━━━━━━━━━━━━━━━━ 20s 57ms/step - accuracy: 0.5993 - loss: 1.0634 - val_accuracy: 0.3135 - val_loss: 1.2698 - learning_rate: 0.0010
Epoch 4/100
228/228 ━━━━━━━━━━━━━━━━━━━━ 21s 58ms/step - accuracy: 0.5999 - loss: 0.9964 - val_accuracy: 0.2639 - val_loss: 1.2525 - learning_rate: 0.0010
Epoch 5/100
228/228 ━━━━━━━━━━━━━━━━━━━━ 15s 57ms/step - accuracy: 0.6011 - loss: 0.9692 - val_accuracy: 0.2639 - val_loss: 1.2208 - learning_rate: 0.0010
Epoch 6/100
228/228 ━━━━━━━━━━━━━━━━━━━━ 15s 57ms/step - accuracy: 0.5889 - loss: 0.9728 - val_accuracy: 0.3135 - val_loss: 1.1890 - learning_rate: 0.0010
Epoch 7/100
228/228 ━━━━━━━━━━━━━━━━━━━━ 15s 59ms/step - accuracy: 0.5977 - loss: 

In [ ]:
# Function to predict
def predict_next_move(moves_sequence):
    # Clean and convert the input moves from strings to integers
    moves_sequence = [int(x.strip('[]')) for x in moves_sequence]  # Convert strings to integers

    # Transform
    moves_sequence_transformed = label_encoder.transform(moves_sequence).reshape(1, -1)

    #kNN initial prediction
    knn_prediction = knn.predict(moves_sequence_transformed).reshape(-1, 1)

    # Combine the original sequence with the KNN prediction
    combined_sequence = np.concatenate([moves_sequence_transformed, knn_prediction], axis=1)

    # Reshape,combined sequence to fit the LSTM model input
    combined_sequence_reshaped = combined_sequence.reshape(1, combined_sequence.shape[1], 1)


    # Make the final prediction using the LSTM model
    predicted_move = model.predict(combined_sequence_reshaped)


    # Convert the predicted class back to the original move label
    predicted_move = label_encoder.inverse_transform([np.argmax(predicted_move)])

    return predicted_move[0]


example_sequence = ['[2]', '[2]', '[2]']  # Moves
predicted_move = predict_next_move(example_sequence)
print(f'Predicted next move: {predicted_move}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step
Predicted next move: 10
